In [8]:
# Import necessary libraries
import pandas as pd
from IPython.display import display

# Load the Excel files into global variables
try:
    factories_data = pd.read_excel("بيانات تسجيل المصانع.xlsx")
    suppliers_data = pd.read_excel("بيانات الموردين.xlsx")
    materials_data = pd.read_excel("المواد.xlsx")
except FileNotFoundError as e:
    raise e

# Initialize a global variable to store selected materials for factories
selected_materials_by_factory = {}

# Function to calculate environmental impact
def calculate_environmental_impact(material_name):
    global materials_data
    material = materials_data[materials_data['اسم المادة'] == material_name]
    if material.empty:
        print(f"المادة {material_name} غير موجودة في البيانات.")
        return None, None

    # Extract environmental data
    carbon_emissions = material['انبعاثات الكربون (كجم/طن)'].values[0]
    waste_impact = material['تأثير إدارة النفايات (1-10)'].values[0]
    water_consumption = material['استهلاك المياه (لتر/طن)'].values[0]

    # Fixed weights
    carbon_weight = 0.5
    waste_weight = 0.3
    water_weight = 0.2

    # Calculate total impact
    total_impact = (
        (carbon_emissions * carbon_weight) +
        (waste_impact * waste_weight) +
        (water_consumption * water_weight)
    )

    # Classify environmental impact
    impact_category = classify_environmental_impact(total_impact)

    return total_impact, impact_category


def classify_environmental_impact(total_impact):
    if total_impact >= 800:
        return "عالي"
    elif total_impact >= 400:
        return "متوسط"
    else:
        return "قليل"

# Function to allow suppliers to add a material
def add_material(supplier_name):
    global materials_data, suppliers_data
    print("\nإضافة مادة جديدة:")
    material_name = input("اسم المادة: ")
    carbon_emissions = float(input("انبعاثات الكربون (كجم/طن): "))
    waste_impact = int(input("تأثير إدارة النفايات (من 1 إلى 10): "))
    water_consumption = float(input("استهلاك المياه (لتر/طن): "))
    license_required = input("هل تتطلب ترخيص؟ (نعم/لا): ").strip().lower()
    license_link = input("رابط الترخيص (إذا كان موجودًا): ") if license_required == "نعم" else None

    # Get supplier location from suppliers_data
    supplier_location = suppliers_data[suppliers_data['اسم المورد'] == supplier_name]['الموقع'].values[0]

    # Append the new material to the materials dataset
    new_material = {
        'اسم المادة': material_name,
        'اسم المورد': supplier_name,
        'الموقع': supplier_location,
        'انبعاثات الكربون (كجم/طن)': carbon_emissions,
        'تأثير إدارة النفايات (1-10)': waste_impact,
        'استهلاك المياه (لتر/طن)': water_consumption,
        'تتطلب ترخيص': "نعم" if license_required == "نعم" else "لا",
        'رابط الترخيص': license_link,
        'المادة البديلة': None,
        'مورد المادة البديلة': None,
        'موقع المورد البديل': None,
        'انبعاثات الكربون للمادة البديلة (كجم/طن)': None,
        'تأثير إدارة النفايات للمادة البديلة (1-10)': None,
        'استهلاك المياه للمادة البديلة (لتر/طن)': None
    }

    # Update materials file
    materials_data = pd.concat([materials_data, pd.DataFrame([new_material])], ignore_index=True)
    with pd.ExcelWriter("المواد.xlsx", mode="a", if_sheet_exists="replace", engine="openpyxl") as writer:
        materials_data.to_excel(writer, index=False)

    # Append the new material to the suppliers dataset
    new_supplier_entry = {
        'اسم المورد': supplier_name,
        'المواد المقدمة': material_name,
        'الموقع': supplier_location,
        'رقم الترخيص الصناعي': "غير متوفر",
        'البريد الإلكتروني': suppliers_data[suppliers_data['اسم المورد'] == supplier_name]['البريد الإلكتروني'].values[0],
        'كلمة المرور': None  # Leave password blank for security
    }
    suppliers_data = pd.concat([suppliers_data, pd.DataFrame([new_supplier_entry])], ignore_index=True)
    
    
    
    
    
# Update suppliers file
    with pd.ExcelWriter("بيانات الموردين.xlsx", mode="a", if_sheet_exists="replace", engine="openpyxl") as writer:
        suppliers_data.to_excel(writer, index=False)

    print(f"\nتمت إضافة المادة {material_name} بنجاح إلى ملف المواد وملف الموردين.")

# Function to show materials provided by a supplier
def show_supplier_materials(supplier_name):
    global materials_data
    supplier_materials = materials_data[materials_data['اسم المورد'] == supplier_name]
    if not supplier_materials.empty:
        print(f"\nالمواد التي يوفرها المورد {supplier_name}:")
        display(supplier_materials[['اسم المادة', 'الموقع', 'تتطلب ترخيص', 'رابط الترخيص']])
        material_name = input("أدخل اسم المادة لعرض التفاصيل: ")
        show_material_details(material_name)
    else:
        print(f"\nلا توجد مواد مرتبطة بالمورد {supplier_name}.")

# Function to show material details
def show_material_details(material_name):
    global materials_data
    material = materials_data[materials_data['اسم المادة'] == material_name]
    if not material.empty:
        print(f"\nتفاصيل المادة {material_name}:")
        display(material[['اسم المادة', 'اسم المورد', 'الموقع', 'تتطلب ترخيص', 'رابط الترخيص', 
                          'انبعاثات الكربون (كجم/طن)', 'تأثير إدارة النفايات (1-10)', 'استهلاك المياه (لتر/طن)']])
    else:
        print(f"لا توجد تفاصيل للمادة {material_name}.")

# Main program logic
def main():
    page, company_name = login_or_signup()
    if page == "factory":
        # Factory operations here
        pass
    elif page == "supplier":
        while True:
            supplier_action = input("\n1- عرض قائمة المصانع\n2- إضافة مادة\n3- عرض المواد التي يوفرها المورد\n4- خروج\nاختر العملية: ")
            if supplier_action == "1":
                print("\nقائمة المصانع:")
                display(factories_data[['اسم المصنع', 'الموقع', 'البريد الإلكتروني']])
            elif supplier_action == "2":
                add_material(company_name)
            elif supplier_action == "3":
                show_supplier_materials(company_name)
            elif supplier_action == "4":
                print("تم تسجيل الخروج.")
                break
            else:
                print("خيار غير صالح.")

# Run the program
main()

1- تسجيل دخول
2- حساب جديد
اختر العملية: 1
1- مصنع
2- مورد
اختر نوع الحساب: 2
اسم الشركة: شركة الإسمنت السعودية
الإيميل: info@saudicement.com
كلمة المرور: Cement123
مرحباً شركة الإسمنت السعودية، تم تسجيل الدخول بنجاح!

1- عرض قائمة المصانع
2- إضافة مادة
3- عرض المواد التي يوفرها المورد
4- خروج
اختر العملية: 2

إضافة مادة جديدة:
اسم المادة: سيراميك
انبعاثات الكربون (كجم/طن): 300
تأثير إدارة النفايات (من 1 إلى 10): 5
استهلاك المياه (لتر/طن): 150
هل تتطلب ترخيص؟ (نعم/لا): لا

تمت إضافة المادة سيراميك بنجاح إلى ملف المواد وملف الموردين.

1- عرض قائمة المصانع
2- إضافة مادة
3- عرض المواد التي يوفرها المورد
4- خروج
اختر العملية: 3

المواد التي يوفرها المورد شركة الإسمنت السعودية:


اسم المادة  الموقع تتطلب ترخيص                            رابط الترخيص
0     الإسمنت  الدمام         نعم  https://mim.gov.sa/mim/services/31380/
8        حديد  الدمام          لا                                     NaN
9    خشب احمر  الدمام         نعم                             sgakjghgh;a
10    سيراميك  الدمام          لا                                    None

أدخل اسم المادة لعرض التفاصيل: سيراميك

تفاصيل المادة سيراميك:


اسم المادة             اسم المورد  الموقع تتطلب ترخيص رابط الترخيص  \
10    سيراميك  شركة الإسمنت السعودية  الدمام          لا         None   

    انبعاثات الكربون (كجم/طن)  تأثير إدارة النفايات (1-10)  \
10                      300.0                            5   

    استهلاك المياه (لتر/طن)  
10                    150.0


1- عرض قائمة المصانع
2- إضافة مادة
3- عرض المواد التي يوفرها المورد
4- خروج
اختر العملية: 4
تم تسجيل الخروج.
